About to rerun this but will alter the way obsids are stored, going to comment out old one and write new one in above it.  Revert if it doesn't work.

In [1]:
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [2]:
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [3]:
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [4]:
olddata = pd.read_csv("n_obj_df.csv")

In [5]:
olddata.head()

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield


In [6]:
testregion = 'Macclesfield'

In [7]:
bucket = []
for obs in olddata[olddata['region_name'] == testregion].obsid.unique():
    d = {}
    d['region_name'] = testregion
    d['Obsid_Used'] = obs
    x = get_outer_polygon_for_obsid(obs)
    area = x.area
    d['Obsid_Area'] = area
    d['polygon_shape'] = x
    bucket.append(d)
results = pd.DataFrame(bucket)

In [8]:
results

,Obsid_Area,Obsid_Used,polygon_shape,region_name
0,10.754545,ESP_012079_0945,"POLYGON ((-67.69881540000002 255.7925763, -68....",Macclesfield
1,2.720774,ESP_011407_0945,"POLYGON ((-79.4964947 242.6512937, -71.7773945...",Macclesfield
2,14.759485,ESP_011723_0945,"POLYGON ((-72.54717990000002 244.8776493, -72....",Macclesfield
3,5.338690,ESP_011934_0945,"POLYGON ((-68.00317840000001 256.1826178, -68....",Macclesfield
4,5.246979,ESP_012501_0945,"POLYGON ((-67.989408 256.1117836, -68.32543100...",Macclesfield
5,6.933368,ESP_012290_0945,"POLYGON ((-66.91617959999999 257.8571416, -67....",Macclesfield
6,7.428567,ESP_011406_0945,"POLYGON ((-66.20375920000001 254.0296356, -66....",Macclesfield
7,6.965716,ESP_021494_0945,"POLYGON ((-66.8270493 258.3039062, -66.9954035...",Macclesfield
8,5.204890,ESP_020782_0945,"POLYGON ((-66.48888240000001 259.2801681, -66....",Macclesfield
9,5.208502,ESP_020716_0945,"POLYGON ((-66.41012389999999 259.1488139, -66....",Macclesfield


In [9]:
bucket = []
for obs in olddata[olddata['region_name'] == testregion].obsid.unique():
    d = {}
    d['obsid'] = obs #we dont want obs, want obs used in combo
    x = get_outer_polygon_for_obsid(obs)
    d['obs_poly'] = x
    bucket.append(d)
testresult = pd.DataFrame(bucket)

In [10]:
df1 = testresult

In [11]:
df2 = interchecker(testresult.obs_poly)
df2.head()
print(len(df2))

16383


In [12]:
len(df1)

14

In [13]:
df1.head()

,obs_poly,obsid
0,"POLYGON ((-67.69881540000002 255.7925763, -68....",ESP_012079_0945
1,"POLYGON ((-79.4964947 242.6512937, -71.7773945...",ESP_011407_0945
2,"POLYGON ((-72.54717990000002 244.8776493, -72....",ESP_011723_0945
3,"POLYGON ((-68.00317840000001 256.1826178, -68....",ESP_011934_0945
4,"POLYGON ((-67.989408 256.1117836, -68.32543100...",ESP_012501_0945


In [14]:
x = 14+91+364+1001+2002+3003+3432+3003+2002+1001+364+91+14+1

In [15]:
print(x) #we expect this many combinations, and we get it! perfect

16383


In [16]:
Testregion_inter = interchecker(results.polygon_shape) #all combos of results

In [17]:
Testregion_inter.head()

,combo_area,elements_used
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68..."


In [18]:
len(Testregion_inter.elements_used) #should be half, elements_used nonunique

16383

In [19]:
results.Obsid_Area.max()

14.759485453249887

In [20]:
Testregion_inter.combo_area.max()

14.759485453249887

In [21]:
Testregion_inter.combo_area.head()

0    0.008637
1    0.073851
2    0.008637
3    0.008637
4    0.008637
Name: combo_area, dtype: float64

In [22]:
bucket2 = []
for ele in Testregion_inter.index:
    #print(ele)
    polylist = Testregion_inter.elements_used[ele]
    bucket = []
    for poly in polylist:
        x = df1.obsid[df1['obs_poly'] == poly].values[0]
        bucket.append(x)
    bucket2.append(bucket)
Testregion_inter['obsids_used'] = np.array(bucket2)


In [23]:
#THIS IS THE REAL ONE! Save!
# bucket2 = []
# for ele in Testregion_inter.index:
#     #print(ele)
#     polylist = Testregion_inter.elements_used[ele]
#     bucket = []
#     for poly in polylist:
#         x = df1.obsid[df1['obs_poly'] == poly].values[0]
#         bucket.append(x)
#     bucket = tuple(bucket)
#     bucket2.append(bucket)
# Testregion_inter['obsids_used'] = np.array(bucket2)


In [24]:
Testregion_inter.head()
#print(Testregion_inter.obsids_used[0])

,combo_area,elements_used,obsids_used
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."


In [25]:
obsdf = Testregion_inter #rename this monstrosity

In [26]:
obsdf.head()

,combo_area,elements_used,obsids_used
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_..."


In [27]:
len(obsdf)

16383

In [28]:
bucket = []
for i in range(len(obsdf.elements_used)): #change to range(len(obsdf.elements_used)) after we check
    #print(len(obsdf.elements_used[i])) #check it's what we want.
    obsdf.at[i, 'number_hirise'] = int(len(obsdf.elements_used[i]))
    #even calling it int(len) still gives float on df, but whatever.

In [29]:
obsdf.head() #remember this still has duplicates atm

,combo_area,elements_used,obsids_used,number_hirise
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",14.0
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0


In [30]:
obsdf.obsids_used[0][0]

'ESP_012079_0945'

In [31]:
obsdf.obsids_used[0]

['ESP_012079_0945',
 'ESP_011407_0945',
 'ESP_011723_0945',
 'ESP_011934_0945',
 'ESP_012501_0945',
 'ESP_012290_0945',
 'ESP_011406_0945',
 'ESP_021494_0945',
 'ESP_020782_0945',
 'ESP_020716_0945',
 'ESP_020571_0945',
 'ESP_020294_0945',
 'ESP_020927_0945',
 'ESP_020242_0945']

In [32]:
bucket2 = []
for obslist in obsdf.obsids_used:
    bucket = []
    for obs in obslist:
        x = olddata.Ls[olddata['obsid'] == obs].values[0]
        bucket.append(x)
    bucket = tuple(bucket)
    bucket2.append(bucket)
obsdf['Ls_used'] = np.array(bucket2)
obsdf['region_name'] = testregion

In [33]:
obsdf.head() #remember this still has duplicates

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",14.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
2,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
3,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
4,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield


In [34]:
obsdf.to_csv('Macclesfield_duplicate_combodf.csv', index=False)

In [35]:
obsdf[obsdf['combo_area'] == obsdf.combo_area.max()]

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
16371,14.759485,"[POLYGON ((-72.54717990000002 244.8776493, -72...",[ESP_011723_0945],1.0,"(198.04,)",Macclesfield


In [36]:
#olddata = pd.read_csv("n_obj_df.csv")
combodf = pd.read_csv("Macclesfield_duplicate_test_combodf.csv")

In [37]:
combodf.head(2)

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
0,0.008637,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",14.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
1,0.073851,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield


In [38]:
obsdf.head(2)

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
0,0.008637,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",14.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
1,0.073851,"[POLYGON ((-67.69881540000002 255.7925763, -68...","[ESP_012079_0945, ESP_011407_0945, ESP_011723_...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield


In [39]:
type(combodf.obsids_used[0]) #combodf is obsdf, read in from csv

str

In [50]:
type(obsdf.obsids_used[0])

list

In [49]:
type(combodf.elements_used[0]) ##combodf is obsdf, read in from csv

str

In [48]:
type(obsdf.elements_used[0])

numpy.ndarray

In [52]:
#np.fromstring(combodf.obsids_used[0], sep = ', ')

In [51]:
#np.fromstring('1 2', sep = ' ')